## Convolution Neural Network

- Mnist data
- 3 convolutional layers
- 2 fully connected layers

### 1. Setting

### 1) Import required libraries

In [9]:
import torch
import torch.nn as nn
import torch.utils as utils
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

#### set Hyperparameters

In [10]:
epoch = 1   #시간이 너무 오래 걸린다...
batch_size = 16
learning_rate = 0.001

#### Download data

In [11]:
# mnist data는 이미 torch에 있다.  target_transform -> target을 바꾸어 주지 않겠다.
mnist_train = dset.MNIST("./",train = True, transform = transforms.ToTensor(), target_transform = None, download = True)
mnist_test  = dset.MNIST("./",train = False, transform = transforms.ToTensor(), target_transform = None, download = True)

#### Check the datasets download

In [12]:
print(mnist_train.__len__())
print(mnist_test.__len__())

# dataset이 잘 받아졌나, 어떤 모양인지 확인하는 작업
img1, label1 = mnist_train.__getitem__(0)
img2, label2 = mnist_test.__getitem__(0)

print(img1.size(), label1)
print(img2.size(), label2)

60000
10000
torch.Size([1, 28, 28]) 5
torch.Size([1, 28, 28]) 7


#### Set Data Loader (input pipeline)

In [13]:
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size = batch_size, shuffle = True)

#### Define Model

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1,16,5), 
                        #mnist는 (28,28,1)이기 때문에 앞에 채널 수와 같은 1을 써줌.
                        #원하는 output의 channel이 16이므로 16, kernal_size를 5로 해준다.
                        # ouput -> batch * (16 * 24 * 24)
                        nn.ReLU(),
                        nn.Conv2d(16,32,5),
                        #conv1을 통과한 채널의 수가 16이므로 ,input은 16,
                        #원하는 output의 채널의 수가 32이므로 32, kernal_size는 5로 해준다.
                        # output -> batch * (32 * 20 * 20)
                        nn.ReLU(),
                        nn.MaxPool2d(2,2)
                        # output -> batch * (32 * 10 * 10)
        )
        
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32,64,5),
                        # output -> batch * (64 * 6 * 6)
                        nn.ReLU(),
                        nn.Conv2d(64,128,5),
                        # output -> batch * (128 * 2 * 2)
                        nn.ReLU()
        )
        
        self.fc = nn.Linear(2*2*128,10)  
        #(채널수 * width * height) 를 input으로 넣고, output을 10으로 맞춰준다.
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(batch_size, -1)
        out = self.fc(out)
        
        return out
    

cnn = CNN().cuda()

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr = learning_rate)

#### Train Model

In [15]:
for i in range(epoch):
    for j,[image,label] in enumerate(train_loader):
        image = Variable(image).cuda()
        label = Variable(label).cuda()
        
        optimizer.zero_grad()
        result = cnn.forward(image)  #cnn(image)해도 똑같이 forward가 진행된다.
        loss = loss_func(result,label)
        loss.backward()
        optimizer.step()
        
        if j % 100 == 0:
            print(loss)

tensor(2.2986, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1561, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.7097, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2327, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6340, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0985, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2395, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1456, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0767, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0548, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0791, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0041, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0047, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0407, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0128, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1119, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0097, device='cuda:0', grad_fn=<NllLossBackward

#### Test Model

In [16]:
cnn.eval()  #dropout하고 batchnormalization 이 부분이 정지 되는 부분 -> 모델을 고정시킨다고 생각하자.
correct = 0
total = 0

for image,label in  test_loader:
    image = Variable(image).cuda()
    result = cnn(image).cuda()
    
    _, predicted = torch.max(result.data, 1)
    total += label.size(0)
    correct += (predicted == label.cuda()).sum()

print("Test Accuracy of the model on the 10000 test image: %f %%" % (100 * correct / total))

Test Accuracy of the model on the 10000 test image: 98.000000 %
